### 문제 : 고객 구매 데이터를 사용해서 고객이 주문한 물품이 제 시간에 도착여부(Reached.on.Time_Y.N) 예측

In [129]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/Train.csv')

In [130]:
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [131]:
df[['Warehouse_block','Mode_of_Shipment']].value_counts()

Warehouse_block  Mode_of_Shipment
F                Ship                2488
D                Ship                1245
C                Ship                1244
B                Ship                1243
A                Ship                1242
F                Flight               592
                 Road                 586
A                Flight               297
D                Flight               297
B                Flight               296
C                Flight               295
A                Road                 294
B                Road                 294
C                Road                 294
D                Road                 292
dtype: int64

In [132]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Warehouse_block'] = le.fit_transform(df['Warehouse_block'])
df['Mode_of_Shipment'] = le.fit_transform(df['Mode_of_Shipment'])
df['Product_importance'] = le.fit_transform(df['Product_importance'])
df['Gender'] = le.fit_transform(df['Gender'])

In [133]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [134]:
x_train, x_test = train_test_split(df, test_size=0.3, stratify=df['Reached.on.Time_Y.N'], random_state=100)

In [135]:
y_train = x_train[['ID','Reached.on.Time_Y.N']]
x_train = x_train.drop('Reached.on.Time_Y.N', axis=1)
y_test = x_test[['ID','Reached.on.Time_Y.N']]
x_test = x_test.drop('Reached.on.Time_Y.N', axis=1)

In [56]:
rf = RandomForestClassifier(random_state=0, n_estimators=100, max_depth=10)
rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=10, random_state=0)

In [156]:
pred = rf.predict(x_test)

In [160]:
pred = pd.DataFrame(pred)[1]
pred

0       0
1       1
2       0
3       1
4       0
       ..
3295    1
3296    1
3297    1
3298    0
3299    0
Name: 1, Length: 3300, dtype: int64

In [136]:
pred_pr = rf.predict_proba(x_test)

In [137]:
pred_pr[1]

array([[6.20252006e-01, 3.79747994e-01],
       [2.82885431e-05, 9.99971711e-01],
       [5.70108259e-01, 4.29891741e-01],
       ...,
       [2.82885431e-05, 9.99971711e-01],
       [5.54347436e-01, 4.45652564e-01],
       [5.87096254e-01, 4.12903746e-01]])

In [145]:
y_test.reset_index(drop=True,inplace=True)

In [146]:
pr = pd.DataFrame(pred_pr[1])

In [147]:
pr.head(10)

,0,1
0,0.620252,0.379748
1,0.000028,0.999972
2,0.570108,0.429892
3,0.000000,1.000000
4,0.567301,0.432699
5,0.540899,0.459101
6,0.580468,0.419532
7,0.570504,0.429496
8,0.000000,1.000000
9,0.000000,1.000000


In [148]:
predd = pd.DataFrame()
predd['Reached.on.Time_Y.N'] = pr[1]
predd['ID'] = y_test['ID']

In [155]:
predd = predd[['ID','Reached.on.Time_Y.N']]
predd

,ID,Reached.on.Time_Y.N
0,8901,0.379748
1,2837,0.999972
2,8494,0.429892
3,1998,1.000000
4,5551,0.432699
...,...,...
3295,1324,1.000000
3296,2501,0.999972
3297,2649,0.999972
3298,10205,0.445653


In [153]:
y_test

,ID,Reached.on.Time_Y.N
0,8901,1
1,2837,1
2,8494,1
3,1998,1
4,5551,1
...,...,...
3295,1324,1
3296,2501,1
3297,2649,1
3298,10205,1


In [161]:
preds = pd.DataFrame()
preds['Reached.on.Time_Y.N'] = pred
preds['ID'] = y_test['ID']

In [163]:
preds

,Reached.on.Time_Y.N,ID
0,0,8901
1,1,2837
2,0,8494
3,1,1998
4,0,5551
...,...,...
3295,1,1324
3296,1,2501
3297,1,2649
3298,0,10205


In [162]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,pred)

ValueError: multiclass-multioutput format is not supported